In [ ]:
import numpy as np
import cv2
import os
from utils import *

In [ ]:
curl_category = ['curl', 'curl_45_out', 'curl_90_out', 'curl_45_palm', 'curl_90_palm']
point_foreward_category = ['point_foreward', 'point_foreward_45_out', 'point_foreward_90_out', 'point_foreward_45_palm', 'point_foreward_90_palm']
point_up_category = ['point_up', 'point_up_out', 'point_up_palm', 'point_up_thumb']
ammo = 30

In [ ]:
def interpret(history, inp):
    if len(history) == 0:
        history.append(inp)
        print('nothing')
        return
    
    last_move = history[-1]
    print('last_move: {} | inp: {}'.format(last_move, inp))
    history.append(inp)
    if inp in curl_category:
        ammo -= 1
        return 'shoot'
    elif inp in point_foreward_category:
        return 'stop'
    elif inp in point_up_category:
        ammo = 30
        return 'reload'
    else:
        print('nothing')
        return

In [ ]:
labels = get_class_labels()

num_values = 100
inps = np.random.choice(labels, num_values)
print(inps)

history = []

for i, inp in enumerate(inps):
    action = interpret(history, inp)
    if action is not None:
        print('{} => {}: do action: {}'.format(len(history) - 2, len(history) - 1, action))
        print('ammo: {}'.format(ammo))
    

In [ ]:
output_dir = 'E:/ar_fps/action_perception/frames'

In [ ]:
cap = cv2.VideoCapture('E:/ar_fps/gameplay.mp4')

In [ ]:
# extract frames

init_box = None
tracker = cv2.TrackerKCF_create()

x_start = 200
y_start = 200
width = 125
height = 125

_, frame = cap.read()
frames = [frame]
# box = cv2.selectROI('Frame', frame, fromCenter=False, showCrosshair=True)
# tracked_boxes = [box]
# tracker = cv2.TrackerKCF_create()
# tracker.init(frame, box)
i = 1
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
       
    orig = frame.copy()
#     ret, box = tracker.update(orig)

#     if ret:
#         x, y, w, h = map(int, box)
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3, 1)
#     else:
#         box = cv2.selectROI('Frame', frame, fromCenter=False, showCrosshair=True)
#         tracker = cv2.TrackerKCF_create()
#         tracker.init(orig, box)
        
        
    frames.append(orig)
#     tracked_boxes.append(box)
        
#     cv2.waitKey(10) & 0xff
#     cv2.imshow('img', frame)
    
    i += 1

cap.release()
cv2.destroyAllWindows()

In [ ]:
# with open('gameplay_data.txt', 'w') as f:
#     for box in tracked_boxes:
        
#         f.write(','.join(list(map(str, box))) + '\n')

In [ ]:
tracked_boxes = []
with open('gameplay_data.txt', 'rb') as f:
    while True:
        line = f.readline()
        if line:
            line = line.decode('utf-8').split('\r\n')[0]
            box = list(map(int, map(float, line.split(','))))
            tracked_boxes.append(box)
        else: 
            break

In [ ]:
from utils import *

In [ ]:
model = load_classifier()

In [ ]:
quit = False
for i, frame in enumerate(frames):
    box = tracked_boxes[i]
    x, y, w, h = map(int, box)
    crop = frame[y:y+h, x:x+w]
    
    # preprocess
    img = cv2.resize(crop, (75, 75))
    batch = np.expand_dims(img, axis=0).astype(np.float32)
    
    # run prediction
    res = model.predict(batch)
    pred_ind = np.argmax(res[0])
    pred = labels[pred_ind]
    print(res)
    print(pred)
    disp = frame.copy()
    cv2.rectangle(disp, (x, y), (x + w, y + h), (0, 255, 0), 3, 1)
    cv2.putText(disp, pred, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1)
    cv2.imshow('img', disp)
    while True:
        k = cv2.waitKey(30) & 0xff
        if k == ord('n'):
            break
        elif k == ord('q'):
            quit = True
            break
        elif k == ord('1'):
            cv2.imwrite(os.path.join(output_dir, 'point_foreward', 'img{}.jpg'.format(i)), crop)
            break
        elif k == ord('2'):
            cv2.imwrite(os.path.join(output_dir, 'point_up', 'img{}.jpg'.format(i)), crop)
            break
        elif k == ord('3'):
            cv2.imwrite(os.path.join(output_dir, 'curl', 'img{}.jpg'.format(i)), crop)
            break
    if quit:
        break
       
cv2.destroyAllWindows()